In [45]:
from scipy.integrate import odeint
import time
import math
import numpy as np
from math import *
from scipy.optimize import newton
from matplotlib import animation, rc
from matplotlib import pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from IPython.display import HTML
import import_ipynb
import Functions_for_modeltest
import warnings
import os

In [46]:
# print(torch.cuda.is_available())  # Sollte True zurückgeben, wenn CUDA korrekt funktioniert.
# print(torch.__version__)  # Zeigt die PyTorch-Version.
# print(torch.cuda.get_device_name(0))  # Gibt den Namen der verwendeten GPU zurück.

In [47]:
# Ignoriere alle UserWarnings
warnings.filterwarnings("ignore", category=UserWarning)

# Ignoriere Warnungen, die "X does not have valid feature names" enthalten
warnings.filterwarnings("ignore", message="X does not have valid feature names")

# Ignoriere Warnungen aus dem sklearn-Modul
warnings.filterwarnings("ignore", module="sklearn")

In [ ]:
theta1 = -20
theta2 = 35
g = 9.8
#this variable sets how strong the angle of the initial conditions should change in case you want differnet initials aswell
#if no initial change desired then set to 0
#Default value is always 1 to implement the condition you set by theta1 and theta2
# amount_of_different_inits = 1
# change_of_angle = 0
# m1_params = [2, 3]
# m2_params = [1, 2]
# l1_params = [1.4, 3]
# l2_params = [1]

#Test combination 2
# m1_params = [1, 2]
# m2_params = [2, 3]
# l1_params = [1]
# l2_params = [1.4, 3]

amount_of_different_inits = 1
change_of_angle = 0
# amount_of_different_inits = 7
# change_of_angle = 5
m1_params = [2]
m2_params = [1]
l1_params = [1.4]
l2_params = [1]

simulation_sets = Functions_for_modeltest.multi_simulations(theta1, theta2, change_of_angle, amount_of_different_inits, m1_params, m2_params, l1_params, l2_params, g)

theta 1 ist -20
theta 2 ist 35
m1 = 2; l1 = 1.4; m2 = 1; l2 = 1

m1 = 2; l1 = 3; m2 = 1; l2 = 1

m1 = 2; l1 = 1.4; m2 = 2; l2 = 1

m1 = 2; l1 = 3; m2 = 2; l2 = 1

m1 = 3; l1 = 1.4; m2 = 1; l2 = 1

m1 = 3; l1 = 3; m2 = 1; l2 = 1

m1 = 3; l1 = 1.4; m2 = 2; l2 = 1

m1 = 3; l1 = 3; m2 = 2; l2 = 1



In [49]:
print(len(simulation_sets))
scaler = MinMaxScaler(feature_range=(-1, 1))
train_datasets, test_datasets = Functions_for_modeltest.data_splitter_and_scaler(scaler, simulation_sets, test_size = 0.2)

#1. Accesses the simulation with the params
#2. chooses the Batches
#3. Acesses the batches and you can choose between 1 (targed_batch) and 0 (training_batch)
print(train_datasets[0][0][1])

8
m1 = 2.0; l1 = 1.4; m2 = 1.0; l2 = 1.0; dt = 0.033288948069241014

m1 = 2.0; l1 = 3.0; m2 = 1.0; l2 = 1.0; dt = 0.033288948069241014

m1 = 2.0; l1 = 1.4; m2 = 2.0; l2 = 1.0; dt = 0.033288948069241014

m1 = 2.0; l1 = 3.0; m2 = 2.0; l2 = 1.0; dt = 0.033288948069241014

m1 = 3.0; l1 = 1.4; m2 = 1.0; l2 = 1.0; dt = 0.033288948069241014

m1 = 3.0; l1 = 3.0; m2 = 1.0; l2 = 1.0; dt = 0.033288948069241014

m1 = 3.0; l1 = 1.4; m2 = 2.0; l2 = 1.0; dt = 0.033288948069241014

m1 = 3.0; l1 = 3.0; m2 = 2.0; l2 = 1.0; dt = 0.033288948069241014

tensor([[-0.4095,  0.8948,  0.1542, -0.7986,  2.0000,  1.0000,  1.4000,  1.0000,
          0.0333]])


In [50]:
class MyLSTM(nn.Module):
    def __init__(self, input_size=4, hidden_size=100, num_layers=1):

        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers)
        self.linear = nn.Linear(hidden_size, input_size)
        self.c_h = (
            torch.zeros(num_layers, 1, hidden_size),
            torch.zeros(num_layers, 1, hidden_size)
        )

    def forward(self, x):
        h, self.c_h = self.lstm(x.view(len(x), 1, -1), self.c_h)
        predictions = self.linear(h.view(len(x), -1))
        return predictions[-1]

    def freeze_layers(self, model, num_layers_to_freeze):
        layer_idx = 0
        for name, param in model.named_parameters():
            if "lstm" in name:
                if layer_idx < num_layers_to_freeze:
                    param.requires_grad = False
                else:
                    param.requires_grad = True
                layer_idx += 1

In [51]:
test1_initials = r'D:\Double Pendulum\Parameterstudy\Test1_initials'
test2_initials = r'D:\Double Pendulum\Parameterstudy\Test2_initials'
test1_params = r'D:\Double Pendulum\Parameterstudy\Test1_params'
test2_params = r'D:\Double Pendulum\Parameterstudy\Test2_params'

test_list_initials = [test1_initials, test2_initials]
test_list_params = [test1_params, test2_params]

In [52]:
#######################################################################################################################################################
## Now all the filepaths to the ML-Models / PINNs
multi_inits_model_paths = r'D:\Double Pendulum\Parameterstudy'

Model_pathlist = [multi_inits_model_paths]
#######################################################################################################################################################

In [53]:
Lagrange_3_3000x1_Anfangsbedingung = 'Lagrange_3_3000x1_Anfangsbedingung'
Energieerhaltung_3_3000x1_Anfangsbedingung = 'Energieerhaltung_3_3000x1_Anfangsbedingung'
Lagrange_7_1000x7_2000x1_Anfangsbedingung = 'Lagrange_7_1000x7_2000x1_Anfangsbedingung'
Energieerhaltung_7_1000x7_2000x1_Anfangsbedingung = 'Energieerhaltung_7_1000x7_2000x1_Anfangsbedingung'
Lagrange_7_1000x7_Anfangsbedingung = 'Lagrange_7_1000x7_Anfangsbedingung'
Energieerhaltung_7_1000x7_Anfangsbedingung = 'Energieerhaltung_7_1000x7_Anfangsbedingung'
Lagrange_3_3000x1_Modellparameter = 'Lagrange_3_3000x1_Modellparameter'
Energieerhaltung_3_3000x1_Modellparameter = 'Energieerhaltung_3_3000x1_Modellparameter'
Lagrange_8_700x8_6000x1_Modellparameter = 'Lagrange_8_700x8_6000x1_Modellparameter'
Energieerhaltung_8_700x8_6000x1_Modellparameter = 'Energieerhaltung_8_700x8_6000x1_Modellparameter'
Lagrange_8_700x8_Modellparameter = 'Lagrange_8_700x8_Modellparameter'
Energieerhaltung_8_700x8_Modellparameter = 'Energieerhaltung_8_700x8_Modellparameter'

In [54]:
Model_list_sorted = [[[Lagrange_3_3000x1_Anfangsbedingung, 3], [Energieerhaltung_3_3000x1_Anfangsbedingung, 3], [Lagrange_7_1000x7_2000x1_Anfangsbedingung, 7], [Energieerhaltung_7_1000x7_2000x1_Anfangsbedingung, 7], [Lagrange_7_1000x7_Anfangsbedingung, 7], [Energieerhaltung_7_1000x7_Anfangsbedingung, 7],
          [Lagrange_3_3000x1_Modellparameter, 3], [Energieerhaltung_3_3000x1_Modellparameter, 3], [Lagrange_8_700x8_6000x1_Modellparameter, 8], [Energieerhaltung_8_700x8_6000x1_Modellparameter, 8], [Lagrange_8_700x8_Modellparameter, 8], [Energieerhaltung_8_700x8_Modellparameter, 8]]]

Model_names = [['Lagrange_3_3000x1_Anfangsbedingung', 'Energieerhaltung_3_3000x1_Anfangsbedingung', 'Lagrange_7_1000x7_2000x1_Anfangsbedingung', 'Energieerhaltung_7_1000x7_2000x1_Anfangsbedingung', 'Lagrange_7_1000x7_Anfangsbedingung', 'Energieerhaltung_7_1000x7_Anfangsbedingung'],
               ['Lagrange_3_3000x1_Modellparameter', 'Energieerhaltung_3_3000x1_Modellparameter', 'Lagrange_8_700x8_6000x1_Modellparameter', 'Energieerhaltung_8_700x8_6000x1_Modellparameter', 'Lagrange_8_700x8_Modellparameter', 'Energieerhaltung_8_700x8_Modellparameter']]


In [ ]:
filepath_excel = test1_params
for i in range(len(Model_pathlist)):
    filepath_model = Model_pathlist[i]
    for model in Model_list_sorted[i]:
        filename_model = model[0]
        model_layers = model[1]
        full_path_model = os.path.join(filepath_model, filename_model)
        print(model_layers)
        print(full_path_model)
        model = MyLSTM(input_size=4, hidden_size=100, num_layers=model_layers)
        model.load_state_dict(torch.load(full_path_model, map_location=torch.device('cpu')))
            
        for j in range(len(test_datasets)):
            testcycle_filename_model = f'test_nr_{j+1}_{filename_model}'
            actual_predictions = Functions_for_modeltest.OneStep(model, model_layers, test_datasets[j], testcycle_filename_model, filepath_excel, steps=100, g=9.81)